In [180]:
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy.sparse.linalg.eigen.arpack as arp



from tenpy.models.model import CouplingModel, MPOModel
from tenpy.models.lattice import Chain
from tenpy.networks.site import BosonSite
from tenpy.networks.mps import MPS
from tenpy.tools.params import asConfig
from tenpy.linalg import np_conserved as npc

### MPO Hamiltonian

In [143]:
class HardCoreBosonModel(CouplingModel,MPOModel):
    
    def __init__(self, model_params):
    
        # 0) read out/set default parameters
        model_params = asConfig(model_params, "BoseModel")
        
        L = model_params.get('L', 12)                 #length of the chain
        t = np.asarray(model_params.get('t', 1.))     #tunnelling rate
        rc = model_params.get('rc', 1.)               #range of soft-shoulder interaction
        V = np.asarray(model_params.get('V', 1.))     # soft-shoulder interaction
        bc_MPS = model_params.get('bc_MPS', 'finite') #boundary condition of MPS
        filling = model_params.get('filling', 0.5)    #average site filling 
        n_max = model_params.get('n_max', 1)          #max number of bosons per site
        conserve = model_params.get('conserve', 'N')  
        
        
        # 3) local physical site
        self.site = BosonSite(Nmax=n_max, conserve=conserve, filling=filling)
        
        # 4) lattice #ARE THESE PARAMETERS OK?
        bc = 'open' if bc_MPS == 'finite' else 'periodic'
        self.lat = Chain(L, self.site, bc=bc, bc_MPS=bc_MPS)
        
        # 5) initialize CouplingModel
        CouplingModel.__init__(self, self.lat)
        
        
        # 6) add terms of the Hamiltonian
        
        #NEAREST NEIGHBOR INTERACTION 
        for u1, u2, dx in self.lat.pairs['nearest_neighbors']:
            self.add_coupling(-t, u1, 'Bd', u2, 'B', dx, plus_hc=True)
            
        #SOFT SHOULDER INTERACTION
        for int_range in range(2,rc+1):
            (u1, u2, dx) = self.lat.find_coupling_pairs(max_dx=L, cutoff=rc).get(int_range)[0]
            self.add_coupling(V, u1, 'N', u2, 'N', dx, plus_hc=False)
        
        # 7) initialize H_MPO
        MPOModel.__init__(self, self.lat, self.calc_H_MPO())
    

In [148]:
#dictionary of model parameters 
model_params = {
    'bc_MPS' : 'finite',          #boundary conditions for MPS
    'conserve' : 'N',             #what should be conserved
    'filling' : 1/2,              #float, average filling
    'L' : 48,                     #length of the Chain
    'n_max' : 1,                  #maximum number of bosons per site
    't' : 1,                      #tunnelling rate
    'V' : 1,                      #potential
    'rc': 4                       #range of soft-shulder ionteraction
}

#initialize the model
model = HardCoreBosonModel(model_params)
 
r"""

Element of the MPO

        p*
        |
        ^
        |
wL -->--W-->--wR
        |
        ^
        |
        p


"""


print(model.H_MPO.get_W(0).get_leg_labels())
print(model.H_MPO.get_W(model_params.get('L')-1).get_leg_labels())

['wL', 'wR', 'p', 'p*']
['wL', 'wR', 'p', 'p*']


### Create MPS

In [149]:
#create MPs according to average filling in model_params
avg_n = model_params.get('filling')

#product state: avg_n full sites and (1-avg_n) empty sites
product_state = ['1']*int(model.lat.N_sites*avg_n) + ['vac']*int(model.lat.N_sites*(1-avg_n))
random.shuffle(product_state)

#create MPS
psi = MPS.from_product_state(model.lat.mps_sites(), product_state, bc=model.lat.bc_MPS)

    
r"""

Element of the MPS
    
    
        
vL -->--B-->--vR
        |
        ^
        |
        p


"""

#print(psi.get_B(0))
#print(psi.get_B(model_params.get('L')-1))

'\n\nElement of the MPS\n    \n    \n        \nvL -->--B-->--vR\n        |\n        ^\n        |\n        p\n\n\n'

### Step 0: Setup
it is imperative to bring the initial MPS into an orthogonal form via a gauge transformation. 

In [150]:
#bring psi into right-orthogonal form
psi.canonical_form()

In [151]:
#function to calculate the initial RP
def initial_RP(i, psi, model, check=False):
    

    #get last term of MPS and remove useless leg
    r"""

    vL-->--B
           |
           ^
           |
           p
    """
    B = psi.get_B(i).squeeze('vR') #vL, p
    if check:print(B.get_leg_labels())


    #create complex conj tensor
    r"""
             p*
             |
             ^
             |
    vL* --<--Bc
    """
    Bc = B.conj() #vL* p*
    if check: print(Bc.get_leg_labels())


    #get last term of MPO and drop useless leg wR
    r"""

            p*
            |
            ^
            |
    wL -->--W
            |
            ^
            |
            p


    """

    W = model.H_MPO.get_W(i).copy()
    W = W.take_slice(1,['wR'])#wL p p*
    if check: print(W.get_leg_labels())

    #create R_(L-1)
    r"""

     vL-->--B
            |
            |
    wL -->--W
            |
            ^
            |
            p


    """
    RP = npc.tensordot(B, W,axes=['p','p*']) #vL wL p
    if check:print(RP.get_leg_labels())

    r"""

     vL-->--B
            |
            |
    wL -->--W
            |  
            |
    vL*--<--B
    """

    RP = npc.tensordot(RP,Bc,axes=['p','p*']) #vL wL vL*
    if check: print(RP.get_leg_labels())

    
    return RP

In [152]:
#function to calculate all the RPs
def update_RP(i, RPs, psi, model, check=False):
    
    #store previous RP
    RP = RPs[i+1]
    
    #MPS term
    r"""

    vL-->--B-->vR
           |
           ^
           |
           p
    """
    B = psi.get_B(i) #vL p vR
    if check:print(B.get_leg_labels())
    
    #complex conj MPS term
    r"""
             p*
             |
             ^
             |
    vL* --<--Bc--<--vR*
    """
    Bc = B.conj() #vL* p* vR*
    if check:print(Bc.get_leg_labels())
        
    #MPO term
    r"""

            p*
            |
            ^
            |
    wL -->--W-->--wR
            |
            ^
            |
            p


    """
    W = model.H_MPO.get_W(i) #wL wR p p*
    if check:print(W.get_leg_labels())
        
    
    #contractions
    
    RP = npc.tensordot(B, RP, axes=['vR','vL']) #vL p [vR], #[vL] wL vL*
    if check:print(RP.get_leg_labels())
        
        
    RP = npc.tensordot(RP, W, axes=[['p','wL'],['p*','wR']]) #vL [p] [wL] vL*, wL [wR] p [p*]
    if check:print(RP.get_leg_labels())
        
    RP = npc.tensordot(RP, Bc, axes=[['p','vL*'],['p*','vR*']]) #vL [vL*] wL [p] ,vL* [p*] [vR*]
    if check:print(RP.get_leg_labels())
        
    return RP


In [210]:
def calculate_all_RPs(psi, model, check=False):
    
    #empty list of RPs
    RPs = [None]*model_params.get('L')
    
    #initial RP
    RP_init = initial_RP(model_params.get('L')-1,psi,model,check=check)
    RPs[model_params.get('L')-1] = RP_init #store in list
    
    for i in range(model_params.get('L')-2,1,-1):
        RP = update_RP(i, RPs, psi, model, check=check)
        RPs[i] = RP
        
    return RPs

In [212]:
RPs = calculate_all_RPs(psi, model, check=False)


### contract the first two MPS sites

In [155]:
def Bij(i,j,psi,check=False):
    
    #first element, squeeze useless leg
    r"""

           Bi-->vR
           |
           ^
           |
           p
    """
    Bi = psi.get_B(i).squeeze('vL') #p vR
    if check: print(Bi.get_leg_labels())
        
        
    #second element
    r"""

    vL-->--Bj-->vR
           |
           ^
           |
           p
    """
    Bj = psi.get_B(j) #vL p vR
    if check: print(Bj.get_leg_labels())
        
        
    #contract on shared bond
    r"""

     -->Bij-->vR
    |    |
    ^    ^
    |    |
    pi  pj
    
    """
    Bij = npc.tensordot(Bi, Bj, axes=['vR','vL'])
    Bij.ireplace_labels([0,1], ['pi','pj'])
    if check: print(Bij.get_leg_labels())
    
    return Bij
    
    
    


In [156]:
contracted_B = Bij(0,1,psi,check=False)

## Step1 :optimization of first bond

In [242]:
def Heff(i,j, psi, model, RPs, check=False):
    #left term MPO, drop useless leg
    r"""

            p*
            |
            ^
            |
            W1-->--wR
            |
            ^
            |
            p


    """
    W1 = model.H_MPO.get_W(i).copy()#wR p p*
    W1 = W1.take_slice(0,['wL'])
    if check: print(W1.get_leg_labels())
        
        
        
    #right term MPO
    r"""

            p*
            |
            ^
            |
    wL -->--W2-->--wR
            |
            ^
            |
            p


    """
    W2 = model.H_MPO.get_W(j) #wL wR p p*
    if check: print(W2.get_leg_labels())
        
        
    #contractions
    RP = RPs[j+1]
    if check:print(RP.get_leg_labels())
        
    Heff = npc.tensordot(W2,RP,axes=['wR','wL']) #wL [wR] p p*, vL [wL] vL*
    if check:print(Heff.get_leg_labels())
    
    Heff = npc.tensordot(W1,Heff,axes=['wR','wL'])#[wR] p p*, [wL] p p* vL vL*
    Heff.ireplace_labels([0,1,2,3],['pi','pi*','pj','pj*'])
    if check:print(Heff.get_leg_labels())
    
    
    return Heff


Heff = Heff(0,1, psi, model, RPs, check=True)

['wR', 'p', 'p*']
['wL', 'wR', 'p', 'p*']
['vL', 'wL', 'vL*']
['wL', 'p', 'p*', 'vL', 'vL*']
['pi', 'pi*', 'pj', 'pj*', 'vL', 'vL*']


In [244]:
#TRY WITH NPC

#reshape Heff
chi = Heff.shape[-2] #vL
d1, d2 = Heff.shape[1], Heff.shape[3] #pi*, pj*

theta_shape = (d1, d2, chi)  #pi pj vL
Heff_shape = (d1 * d2 * chi, d1 * d2 * chi)

theta0 = psi.get_theta(0) #vl p0 p1 vR
theta0 = theta0.combine_legs([[0,2],[1,3]])
print(theta0.get_leg_labels())


#LanczosGroundState(Heff, psi0, options, orthogonal_to=[])

['(vL.p1)', '(p0.vR)']


In [232]:
#ONLY VALID FOR THE FIRST TWO SITES

#reshape Heff
chi = Heff.shape[-2] #vL
d1, d2 = Heff.shape[1], Heff.shape[3] #pi*, pj*

theta_shape = (d1, d2, chi)  #pi pj vL
Heff_shape = (d1 * d2 * chi, d1 * d2 * chi)


# Diagonalize Heff, find ground state `theta`
theta0 = np.reshape(psi.get_theta(0).to_ndarray(), theta_shape)  # initial guess
Heff = np.reshape(Heff.to_ndarray(),Heff_shape)

e, v = arp.eigsh(Heff, k=1, which='SA', return_eigenvectors=True, v0=theta0)
theta_new = np.reshape(v[:, 0], theta_shape)

In [233]:
print(theta_new.shape)

(2, 2, 1)


### Step 1b: Adaptive Restoration of MPS Form

In [240]:
from scipy.linalg import svd

def SVD_initial(theta, chi_max, eps):
    r"""
    Split a two-site wave function as follows::
               (theta)-- vR     =>         (A)--diag(S)--(B)-- vR
                |   |                       |             |
                i   j                       i             j
    """
    #store dimensions
    dL, dR, chivR = theta.shape #pi pj vR
    
    #reshape as a matrix
    theta = np.reshape(theta, [dL, dR * chivR])
    
    #decompose according to SVD
    X, Y, Z = svd(theta, full_matrices=False)
    
    # truncate
    #truncation dimension, minimum between chi_max and all the singular values greater than eps
    chivC = min(chi_max, np.sum(Y > eps)) 
    
    # keep the largest `chivC` singular values
    piv = np.argsort(Y)[::-1][:chivC]  
    X, Y, Z = X[:, piv], Y[piv], Z[piv, :]
    
    # renormalize
    S = Y / np.linalg.norm(Y)  # == Y/sqrt(sum(Y**2))
    
    # split legs of X and Z
    A = np.reshape(X, [dL, chivC]) #vL, i, chivC
    B = np.reshape(Z, [chivC, dR, chivR]) #chivC, j, vR
    
    
    return A, S, B
    
    
    
    
Ui, Sij, Vj = SVD_initial(theta_new,chi_max=30, eps=1.e-10)


print(type(Ui),Ui.shape)
print(type(Sij),Sij.shape)
print(type(Vj),Vj.shape)


<class 'numpy.ndarray'> (2, 2)
<class 'numpy.ndarray'> (2,)
<class 'numpy.ndarray'> (2, 2, 1)


In [ ]:
#Absorb S into V
